Reviews Extracting Using Webscraping

In [1]:
#Importing Libraries

from selenium import webdriver
from time import sleep
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from sklearn.metrics import classification_report
import pandas as pd
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix
nltk.download('punkt')

from nltk.corpus import stopwords

nltk.download('stopwords')
import string
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url = "https://www.imdb.com/title/tt15398776/reviews/?ref_=tt_ql_2"

l=[]

def get_cpf():
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')  # Optional: Run Firefox in headless mode
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    scrolls = 15  # Change the number of scrolls as needed
    for i in range(scrolls):
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
        time.sleep(2)
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "title")))
        elements = driver.find_elements(By.CLASS_NAME, "title")
        for element in elements:

            l.append(element.text)
        driver.find_element("id",'load-more-trigger').click()
        sleep(2)
    driver.quit()

#Labels as per Reviews
    columns = ['Reviews']
    df = pd.DataFrame(l,columns=columns)
    print(df)
    sent = SentimentIntensityAnalyzer()
    polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in df['Reviews']]
    df['sentiment_score'] = polarity

    def senti(sent):
        if sent >= 0.05:
            return 'Positive'
        elif sent >= -0.05 and sent < 0.05:
            return 'Neutral'
        else:
            return 'Negative'
    df['Sentiment'] = df['sentiment_score'].apply(senti)
    print(df.head())
    df.drop('sentiment_score',axis=1,inplace=True)
    df.drop_duplicates(inplace=True)
    print(df)

#Preprocessing Extracted Text

    def transform_text(text):
        text = text.lower()
        text = nltk.word_tokenize(text)
        y = []
        for i in text:
            if i.isalnum():
                y.append(i)

        text = y[:]
        y.clear()
        for i in text:
            if i not in stopwords.words('english') and i not in string.punctuation:
                y.append(i)
        text = y[:]
        y.clear()

        for i in text:
            y.append(ps.stem(word=i))

        return " ".join(y)


    df['Reviews_Cleaned']=df['Reviews'].apply(transform_text)

#Converting df to csv

    df.to_csv("Film_reviews.csv")

    from google.colab import drive
    drive.mount('/content/drive')

    !mv Film_reviews.csv '/content/drive/My Drive/'



get_cpf()

                                                Reviews
0     A challenging watch to be sure, but a worthwhi...
1                                 Murphy is exceptional
2                                Quality but exhausting
3     A brilliantly layered examination of a man thr...
4     Nolan delivers a powerfull biopic that shows t...
...                                                 ...
3001            A cinematic masterpiece in every aspect
3002                       Everything about Oppenheimer
3003             Masterpiece isn't a strong enough word
3004     Makes up for that mess of a movie called Tenet
3005                                                   

[3006 rows x 1 columns]
                                             Reviews  sentiment_score  \
0  A challenging watch to be sure, but a worthwhi...             0.62   
1                              Murphy is exceptional             0.00   
2                             Quality but exhausting            -0.50   
3  A brilli

In [3]:
#Reading the csv

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Film_reviews.csv')
print(df)

     Unnamed: 0                                            Reviews Sentiment  \
0             0  A challenging watch to be sure, but a worthwhi...  Positive   
1             1                              Murphy is exceptional   Neutral   
2             2                             Quality but exhausting  Negative   
3             3  A brilliantly layered examination of a man thr...  Positive   
4             4  Nolan delivers a powerfull biopic that shows t...   Neutral   
..          ...                                                ...       ...   
364        3000      A "very serious" film for the ADHD generation  Negative   
365        3001            A cinematic masterpiece in every aspect  Positive   
366        3002                       Everything about Oppenheimer   Neutral   
367        3003             Masterpiece isn't a strong enough word  Positive   
368        3004     Makes up for that mess of a movie called Tenet  Negative   

                                       

In [5]:
df.isnull().sum()

Unnamed: 0         0
Reviews            1
Sentiment          0
Reviews_Cleaned    4
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.drop(['Unnamed: 0','Reviews'],axis=1,inplace=True)
df.head()

,Sentiment,Reviews_Cleaned
0,Positive,challeng watch sure worthwhil one
1,Neutral,murphi except
2,Negative,qualiti exhaust
3,Positive,brilliantli layer examin man throughout incred...
4,Neutral,nolan deliv powerful biopic show dark side hum...


In [8]:
df['Sentiment_1'] = df['Sentiment'].factorize()[0]
df.head()

,Sentiment,Reviews_Cleaned,Sentiment_1
0,Positive,challeng watch sure worthwhil one,0
1,Neutral,murphi except,1
2,Negative,qualiti exhaust,2
3,Positive,brilliantli layer examin man throughout incred...,0
4,Neutral,nolan deliv powerful biopic show dark side hum...,1


In [9]:
X=list(df['Reviews_Cleaned'])

In [10]:
y=list(df['Sentiment_1'])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [14]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, y_train)
test_dataset = IMDbDataset(test_encodings, y_test)

In [15]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=15,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=3)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
10,1.096900
20,1.101600
30,1.099300
40,1.087300
50,1.091700
60,1.077200
70,1.069800
80,1.026600
90,1.013800
100,0.983700


TrainOutput(global_step=285, training_loss=0.5914179316084636, metrics={'train_runtime': 931.3001, 'train_samples_per_second': 4.703, 'train_steps_per_second': 0.306, 'total_flos': 32863884854040.0, 'train_loss': 0.5914179316084636, 'epoch': 15.0})

In [16]:
trainer.evaluate(test_dataset)

{'eval_loss': 1.6919312477111816,
 'eval_runtime': 3.3166,
 'eval_samples_per_second': 22.011,
 'eval_steps_per_second': 0.603,
 'epoch': 15.0}

In [17]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.7296746 , -2.0112948 , -1.8526702 ],
       [-2.468015  ,  1.0145892 ,  1.0706868 ],
       [-1.5206841 , -2.4638333 ,  3.8076947 ],
       [-2.3076854 ,  4.017075  , -2.4590907 ],
       [ 3.5134392 , -1.5113349 , -2.0067184 ],
       [-0.5229289 , -0.14089338,  0.34152076],
       [-2.3973708 ,  4.0713377 , -2.4944425 ],
       [-1.4626722 , -2.4826436 ,  3.7952662 ],
       [-2.0878644 ,  3.9427614 , -2.6338944 ],
       [-1.9731612 ,  3.9647527 , -2.686778  ],
       [ 2.8162522 , -0.2670702 , -2.774788  ],
       [ 4.2814417 , -2.435521  , -2.0034306 ],
       [ 0.988691  ,  1.5742828 , -2.8562884 ],
       [ 1.8267406 , -3.1495614 ,  1.3197623 ],
       [-2.417234  ,  4.0453672 , -2.4260254 ],
       [ 4.0945864 , -2.4401302 , -1.78017   ],
       [ 3.852376  , -1.5170922 , -2.5083807 ],
       [-1.7840635 , -1.8232695 ,  3.4584    ],
       [ 4.261289  , -2.431888  , -2.0190222 ],
       [-2.4105754 ,  4.113406  , -2.414767  ],
       [-1.

In [18]:
trainer.predict(test_dataset)[1].shape

(73,)

In [19]:
output=trainer.predict(test_dataset)[1]

In [20]:
cm=confusion_matrix(y_test,output)
cm

array([[39,  0,  0],
       [ 0, 23,  0],
       [ 0,  0, 11]])

Predicting New Review

In [21]:
def predict_new_string(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class probabilities
    logits = outputs.logits

    # Get predicted class label
    predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class

In [22]:
new_text = "The film is overrated , not worth watching"
predicted_class = predict_new_string(new_text)
if predicted_class==0:
  print("Review is Positive")
elif predicted_class==1:
  print("Review is Neutral")
elif predicted_class==2:
  print("Review is Negative")

Review is Negative
